<a href="https://colab.research.google.com/github/Rusildur/Pipeline_module/blob/main/pipeline_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Расчетный модуль для трубопроводов

## Исходные данные для всех расчетов и допущения

In [ ]:
from math import *
import numpy as np
import pandas as pd 
import openpyxl as ox

Авторизуемся для соединения Colab c Drive для чтения sheets

Таблицы с сведениями по веществам

https://docs.google.com/spreadsheets/d/1eOAJ_aZ6n5LLz7yxdIRQMLoxkY5uZdpgfizijixzC0M/edit?usp=sharing


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
wb = gc.open('pipeline_bd')

In [ ]:
rows_agent = wb.sheet1.get_all_values()

In [ ]:
df_agents = pd.DataFrame(rows_agent[1:], columns=rows_agent[0])

In [ ]:
df_agents.head()

,параметры,нефть,бензин,дт,метан,шфлу,бгс,пнг,конденсат,пропан
0,Плотность,900,750,850,"0,717",680,690,"0,997",680,528
1,Тип,ЛВЖ,ЛВЖ,ГЖ,ГФ,СУГ,ЛВЖ,ГФ,ЛВЖ,СУГ
2,Твсп,-25,-39,35,-200,-30,-39,-200,-100,-30
3,Иисп_с,"3,396","0,19","0,0175",,"3,17","0,19",,"0,19","3,17"
4,Иисп_в,"0,00009","0,00009","0,00009",,,"0,00009",,"0,00009",


In [ ]:
P_0 = 101000  #атмосферное давление ПА
ro = 890  # плотность нефти кг/м3   ## тестово данные подаются вручную, дальше будет разработана функция выбора вещества из БД
ro = ro/1000

Время остановки напорного режима

In [ ]:
t_sv = 21600
t_m = 1800
t_g = 360
t_acf = 3600

###Частота иницирующих событий

Выбор значений частот возникновения событий, инициирующих аварии произведен на основе обобщенных статистических данных и мнения экспертов. Учитывая, что в настоящее время отсутствует нормализованный механизм по сбору статистики отказов оборудования, при использовании статистических данных из литературных источников следует оценивать степень их достоверности и понимать, что такие данные, как правило, дают лишь порядок величины. 

В данной работе при оценке риска используются сведения, рекомендованные приказом Ростехнадзора № 387 от 03.11.2022 
https://drive.google.com/file/d/1VJbsNr1Mxuxcyt2jCvMhBAPyHM8Lqheh/view?usp=share_link

In [ ]:
wb_freq = gc.open("pipeline_bd").get_worksheet(2)

In [ ]:
rows_freq = wb_freq.get_all_values()

In [ ]:
df_freq = pd.DataFrame(rows_freq[1:], columns=rows_freq[0])

In [ ]:
df_freq.head()

,оборудование,частичная,полная
0,труба_75,"1,00E-06","5,00E-06"
1,труба_75_150,"3,00E-07","2,00E-06"
2,труба_150,"1,00E-07","5,00E-07"


In [ ]:
df_freq['частичная'] = df_freq['частичная'].str.replace(',', '.').astype(float)
df_freq['полная'] = df_freq['полная'].str.replace(',', '.').astype(float)

In [ ]:
df_freq

,оборудование,частичная,полная
0,труба_75,1.000000e-06,5.000000e-06
1,труба_75_150,3.000000e-07,2.000000e-06
2,труба_150,1.000000e-07,5.000000e-07


In [ ]:
#df_freq = pd.DataFrame({'оборудование':['труба_75', 'труба_75_150', 'труба_150	'],                  'частичная':[1.0e-06,3.0e-07,1.000000e-07],                 'полная':[5.e-06, 2.000000e-06, 5.000000e-07]                  })

In [ ]:
df_freq

,оборудование,частичная,полная
0,труба_75,1.000000e-06,5.000000e-06
1,труба_75_150,3.000000e-07,2.000000e-06
2,труба_150,1.000000e-07,5.000000e-07


### Вероятность возникновения сценариев аварий (test)

In [ ]:
scenario_lvz = {'полная': {
                'пролив': 0.608, 
                'пожар': 0.2,
                'вспышка': 0.0768,
                'ТВС': 0.1152}, 
                'частичная': {
                'пролив': 0.93026, 
                'пожар': 0.035,
                'вспышка': 0.026402,
                'ТВС': 0.0083376}}                 

# Функции для обработки данных

##Подбора веществ

Для поиска в датафрейме/сводной таблице/базе данных сведений по веществу

Входные параметры: вещество - "agent" и оборудование -  "facilities".

In [ ]:
def choice_agent():
  pass 


##Объем трубопровода


In [ ]:
def Volume_pipeline(dl, diam, st):
    """"Функция расчета объема трубопровода/
    Принимает длину в метрах, диаметр и толщину стенки в мм"""
    radius = (diam/2 - st)/1000     #находим радиус и переводим в метры
    V = pi * dl * radius ** 2
    return V

##Напор трубопровода 

In [ ]:
def flow_pipe(P, S):
  P = P * 1000000
  U = sqrt( 2 * (P - P_0) / ro * 1000)
  M_0 = 0.6 * S * U * ro 
  return M_0

##Площадь пролива

In [ ]:
def Strait_area(massa, ro,  koef_str):
    """"Функция расчета площади пролива на неограниченное пространство
    Принмает массу, плотность вещества т/м3 и коэффициент разлития"""
    V_mass = massa / ro
    S = V_mass * koef_str
    return S

##Площадь отверстий

In [ ]:
def hole_area(diam, st, type = 'малая'):
  if type == 'свищ':
    return 0.0001
  elif type == 'гильотина':
    radius = (diam/2 - st)/1000
    S = pi * radius ** 2
    res = S *  0.179   
    return res 
  else:
    radius = (diam/2 - st)/1000
    S = pi * radius ** 2
    res = S * 0.0072
    return res 

##Количество вещества в трубопроводах


In [ ]:
def substanc_pipe(q_napor, q_beznapor, t_obnr, t_zadv, t_acf ):
  M1 = q_napor * t_obnr
  M2 = q_beznapor * t_zadv
  M3 = q_beznapor * t_acf
  return (M1 + M2 + M3) / 1000

##Интенсивность испарения

In [ ]:
def Evaporation_404():
    """"Интенсивность испарения для нефти 
    кг/м2час????"""
    M = 200     #  Молярная масса нефти по расчетной Антона
    P_n = press_antuan(20, - 25)
    W = 10 ** -6 * sqrt(M) * P_n
    return W 

##Давление насыщенных паров (test)

Давление насыщенных паров через температуры

In [ ]:
def press_antuan(t_r, t_vsp ):
  P_n = exp(6.908 + 0.0433 * (t_r - 0.924 * t_vsp + 2.055) ) / (1047 + 7.48 * t_vsp)
  return P_n

Давление насыщенных паров Антуан

In [ ]:
def press_temp(t_r):
  A = 15.6782
  B = 2154.9
  C = -34.42
  P_n = 10 ** (A - B / (t_r + C) )
  return P_n

##Частота иницирующего события (test)

In [ ]:
def frequencies_pipline(diam):
  result = {'полная': '-' ,'частичная': '-'  }
  if diam < 75:
    result['полная'] = df_freq['полная'][0]
    result['частичная'] = df_freq['частичная'][0]
  elif 75 <= diam <= 150:
    result['полная'] = df_freq['полная'][1]
    result['частичная'] = df_freq['частичная'][1]    
  elif diam > 150:
    result['полная'] = df_freq['полная'][2]
    result['частичная'] = df_freq['частичная'][2] 
  else:
    return 'ERROR'

  return result 



##Частота реализации сценариев

In [ ]:
def scenarios():
  pass


##Классификация режимов сгорания облака

In [ ]:
def classification_cloud():
  pass 

#Функции расчета зон поражения

##Взрыв топливно-воздушной смеси (ТВС)

In [ ]:
def TVS_404(M_t):
    """ Функция расчета взрыва ТВС
    принимает массу испарившихся веществ
    """
    result = {'100': '-','53': '-', '28': '-', '12': '-','5': '-','3': '-'}
    betta = 1.14   # коэффициент для нефти
    E = M_t * 44 * 10 ** 6 * betta      # Удельное энерговыделение
    u = 43 * M_t ** (1/6)          # дефлаграция скорости фронта пламени для 5 класса
    x = 0.1
    while True:
        Rx = x / ((E / P_0) ** (1/3))
        if Rx < 0.34:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/0.34 - 0.14/0.34 ** 2)
        else:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/Rx - 0.14/Rx ** 2)
        
        delta_P = Px1 * 101
        if round(delta_P,2) == 100.0:
          result['100'] = round(x,2)
        elif round(delta_P,2) == 53.0:
          result['53'] = round(x,2)
        elif round(delta_P,2) == 28.0:
          result['28'] = round(x,2)
        elif round(delta_P,2) == 12.0:
          result['12'] = round(x,2)   
        elif round(delta_P,2) == 5.0:
          result['5'] = round(x,2)        
        elif round(delta_P,2) == 3.0:
          result['3'] = round(x,2) 
          return result 
          break 
        elif delta_P < 1:
          return result 
          break
        x += 0.1

##Пожар-вспышка

In [ ]:
def fire_flash_404(m, ro_g = 10, C_nkpr = 1.1):
    """" Модуль расчета пожара вспышки.
    Принимает массу, плотность испарившейся нефти, Нижний концетрационный предел нефти"""
    ro_g = 10       # плотность ГГ нефти
    C_nkpr = 1.1        # нижний концентрационный предел распространения пламени нефти
    R_f = 1.2 * 7.8 * (m / (ro_g * C_nkpr))** 0.33
    return R_f

##Пожар пролива

In [ ]:
def StraitFire_404(S):
    """" Функция для расчета пожара пролива (!!! пока только нефть).
    В тестовом варианте принимает значение площади пролива S. 
    Следует дополнить следующими входными данными:
    - вещество (молярная масса, плотность насыщенных паров, плотность )
    - скорость ветра
    - учесть в функции разный расчет Ef для других видов веществ
    """
    result = {'10,5': '-', '7,0': '-', '1,4': '-'}
    d = sqrt(4 * S / pi)    #эффективный диаметр пролива
    m_u = 0.04          #удельная скорость выгорания топлива
    ro_parov = 218/22.4     #плотность насыщенных паров при испарении
    p_okr_pro = 1.2250      #плотность окружающего воздуха 
    w_0 = 0             #скорость ветра
    Ef = 140 + exp(-0.12 * d) + 20 * (1 - exp(-0.12 * d))       # среднеповерхностная интенсивность теплового излучения пламени
    theta = 0       # угол отклонения пламени от вертикали под действием ветра. (принимается равной 0, если не учитываем ветер)
    u_n = w_0 / np.cbrt(m_u * 9.81 * d / ro_parov)
    if u_n < 1:
        theta = 0
    else:
        theta = u_n ** (-0.5)
    theta = 0 
    
    if u_n >= 1:
        L = 55 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.67 * u_n ** 0.21
    else:
        L = 42 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.61
    x = 1
    while True:
        a = 2 * L / d
        b = 2 * x / d
        A = sqrt((a**2 + (b + 1)**2 - 2 * a * (b + 1) * sin(theta)))
        B = sqrt((a**2 + (b - 1)**2 - 2 * a * (b - 1) * sin(theta)))
        C = sqrt(1 + (b**2 - 1) * cos(theta)**2)
        D = abs((b - 1) / (b + 1)) ** 0.5
        E = a * cos(theta) / (b - a * sin(theta))
        F = abs(b**2 - 1) ** 0.5
        F_v = 1/pi * ( - E * atan(D) + E * ((a**2 + (b+1)**2 - 2*b * (1 + a * sin(theta)))/ (A * B)) * atan(A * D / B) + cos(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C)) * atan(F**2 * sin(theta)/ (F * C))))
        F_n = 1/pi * (atan(1/D) + sin(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C) ) + atan(F**2 * sin(theta) / (F * C))) - ((a**2 + (b+1)**2 - 2 *(b+1+a*b*sin(theta)))/(A * B)) * atan(A*D/B))
        F_q = sqrt(F_v**2 + F_n**2)
        ta = exp(-7 * 10 ** -4 * (x - 0.5 * d))
        q = Ef * F_q * ta
        if round(q, 2) == 10.5:
          result['10,5'] = round(x, 2)
        elif round(q) == 7:
          result['7,0'] = round(x, 2)
        elif round(q, 2) == 1.4:
          result['1,4'] = round(x, 2)
          return result 
          break 

        if q < 1:
          return result
          break
        x += 0.1


#Чтение и обработка данных из Excel

##считываем данные из заполненной таблицы Excel

In [ ]:
!gdown --id 1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa
To: /content/test.xlsx
100% 107k/107k [00:00<00:00, 63.0MB/s]


In [ ]:
df = pd.read_excel('test.xlsx', index_col='№', sheet_name = 'Oborud')

In [ ]:
df.head(3)

,наименование оборудования,Тип оборудования,Вещество,Тип содержимого,"Количество опасного вещества, т",Температура,давление,"Интенсивность прокачки, м3/сут",время перекрытия,"Протяженность, м",диаметр,толщина стенки,Коэффициент подстил поверхности,Класс загроможденности пространства,Блок,№ в блоке,частота обхода (дней),пересечения - водотоки,пересечения - ЖД,"пересечения - АД, категория дорог"
№,,,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",труба,нефть,ЛВЖ,NaN,-20..20,4,26.0,300,10000,1000,12,5,4,Система промысловых трубопроводов,1,3,NaN,NaN,NaN
2,"нефтепровод (1000х12, 5000 м)",труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,5000,1000,12,5,4,Система промысловых трубопроводов,2,3,1.0,NaN,NaN
3,"нефтепровод (1000х12, 1000 м)",труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,1000,1000,12,5,4,Система промысловых трубопроводов,3,3,1.0,NaN,NaN


Чтение и получение коэффициентов из исходного файла Excel

In [ ]:
wb = ox.load_workbook('test.xlsx')

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
wb_info = wb['Info']

In [ ]:
koef_eco_water = {'Квг': wb_info['B3'].value, 
                  'Кдл': wb_info['B4'].value,
                  'Кв': wb_info['B5'].value,
                  'Кин': wb_info['B6'].value}

In [ ]:
koef_eco_soil = {'СХВ': wb_info['D3'].value, 
                  'Кг': wb_info['D4'].value,
                  'Кисп': wb_info['D5'].value,
                  'Тх': wb_info['D6'].value}

In [ ]:
koef_eco_vapor= {'Нпл': wb_info['F3'].value, 
                  'Кот': wb_info['F4'].value,
                  'Кср': wb_info['F5'].value,
                  'Кг': wb_info['F6'].value}

In [ ]:
koef_eco_fire= {'Тх': wb_info['H3'].value, 
                  'Кср': wb_info['H4'].value,
                  'Кг': wb_info['H5'].value}

In [ ]:
bd_limit = {'max_smena': wb_info['B9'].value, 
                'speed_inspector': round(wb_info['B12'].value / 3.6,2)}

In [ ]:
bd_limit['speed_inspector']

2.78

##Расчитываем объем и массу по каждому участку трубопровода

In [ ]:
df['объем'] = round(Volume_pipeline(df['Протяженность, м'], df['диаметр'], df['толщина стенки']), 2)

In [ ]:
df.head(1)

,наименование оборудования,Тип оборудования,Вещество,Тип содержимого,"Количество опасного вещества, т",Температура,давление,"Интенсивность прокачки, м3/сут",время перекрытия,"Протяженность, м",...,толщина стенки,Коэффициент подстил поверхности,Класс загроможденности пространства,Блок,№ в блоке,частота обхода (дней),пересечения - водотоки,пересечения - ЖД,"пересечения - АД, категория дорог",объем
№,,,,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",труба,нефть,ЛВЖ,NaN,-20..20,4,26.0,300,10000,...,12,5,4,Система промысловых трубопроводов,1,3,NaN,NaN,NaN,7481.51


In [ ]:
df['масса_в_участке'] = round(df['объем'] * ro, 2)

##Масса вещества участвующего в аварии и площади пролива

In [ ]:
preparation_df = pd.DataFrame(columns=['полная','частичная','частичная_малая', 'частичная_свищ','площадь полная', 'площадь частичная', 'облако', 'твс', 
                                       'напор гильотины', 'напор малой', 'напор свища',
                                       'безнапор гильотины', 'безнапор малой', 'безнапор свища'])

In [ ]:
preparation_df.insert(0,'наименование оборудования', df['наименование оборудования'])

Расчитываем площади отверстий

In [ ]:
preparation_df['площадь_гильотины'] = hole_area(df['диаметр'], df['толщина стенки'], type='гильотина')

In [ ]:
preparation_df['площадь_малой'] = hole_area(df['диаметр'], df['толщина стенки'], type='малая')

In [ ]:
preparation_df['площадь_свища'] = hole_area(df['диаметр'], df['толщина стенки'], type='свищ')

Расчитываем напорный и безнапорный режим для гильотины и малой трещины

In [ ]:
for i, r in df.iterrows():
  preparation_df.loc[i, 'напор гильотины'] = flow_pipe(df['давление'][i], preparation_df['площадь_гильотины'][i])
  preparation_df.loc[i, 'напор малой'] = flow_pipe(df['давление'][i], preparation_df['площадь_малой'][i])
  preparation_df.loc[i, 'безнапор гильотины'] = flow_pipe(1.2, preparation_df['площадь_гильотины'][i])
  preparation_df.loc[i, 'безнапор малой'] = flow_pipe(1.2, preparation_df['площадь_малой'][i])
  preparation_df.loc[i, 'напор свища'] = flow_pipe(df['давление'][i], preparation_df['площадь_свища'][i])
  preparation_df.loc[i, 'безнапор свища'] = flow_pipe(1.2, preparation_df['площадь_свища'][i])

Расчитываем массы

In [ ]:
masses_cv_gil_df = pd.DataFrame(columns = ['V1_свищ', 'V2_свищ', 'V3_свищ',
                                           'V1_малая', 'V2_малая', 'V3_малая', 
                                        'V1_гильотина', 'V2_гильотина', 'V3_гильотина',
                                        'V3_свищ_АСФ', 'V3_малая_АСФ',  'V3_гильотина_АСФ' ])

In [ ]:
masses_cv_gil_df.insert(0,'наименование оборудования', df['наименование оборудования'])

In [ ]:
for i, r in df.iterrows():
   masses_cv_gil_df.loc[i, 'V1_свищ'] = round(preparation_df.loc[i, 'напор свища'] * df.loc[i, 'частота обхода (дней)'] * 86400 / 1000, 4)
   masses_cv_gil_df.loc[i, 'V1_малая'] = round(preparation_df.loc[i, 'напор малой'] * df.loc[i, 'частота обхода (дней)'] * 86400 / 1000, 4)
   masses_cv_gil_df.loc[i, 'V1_гильотина'] = round(preparation_df.loc[i, 'напор гильотины'] * df.loc[i, 'время перекрытия']/ 1000, 4)
   #V2
   masses_cv_gil_df.loc[i, 'V2_свищ'] = round(preparation_df.loc[i, 'безнапор свища'] * df.loc[i, 'время перекрытия'] / 1000, 4)
   masses_cv_gil_df.loc[i, 'V2_малая'] = round(preparation_df.loc[i, 'безнапор малой'] * df.loc[i, 'время перекрытия'] / 1000, 4)
   masses_cv_gil_df.loc[i, 'V2_гильотина'] = round(preparation_df.loc[i, 'безнапор гильотины'] * df.loc[i, 'время перекрытия'] / 1000, 4)
   #V3
   masses_cv_gil_df.loc[i, 'V3_свищ'] = round(preparation_df.loc[i, 'безнапор свища'] * (df.loc[i, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000, 4)
   masses_cv_gil_df.loc[i, 'V3_малая'] = round(preparation_df.loc[i, 'безнапор малой'] * (df.loc[i, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000, 4)
   masses_cv_gil_df.loc[i, 'V3_гильотина'] = round(preparation_df.loc[i, 'безнапор гильотины'] * (df.loc[i, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000, 4)
   #V3 АСФ
   masses_cv_gil_df.loc[i, 'V3_свищ_АСФ'] = round(preparation_df.loc[i, 'безнапор свища'] * 3600 / 1000, 4)
   masses_cv_gil_df.loc[i, 'V3_малая_АСФ'] = round(preparation_df.loc[i, 'безнапор малой'] * 3600 / 1000, 4)
   masses_cv_gil_df.loc[i, 'V3_гильотина_АСФ'] = round(preparation_df.loc[i, 'безнапор гильотины'] * 3600 / 1000, 4)

In [ ]:
df['масса_в_участке'].head(3)

№
1    6658.54
2    3329.28
3     665.85
Name: масса_в_участке, dtype: float64

In [ ]:
masses_cv_gil_df.head(1)

,наименование оборудования,V1_свищ,V2_свищ,V3_свищ,V1_малая,V2_малая,V3_малая,V1_гильотина,V2_гильотина,V3_гильотина,V3_свищ_АСФ,V3_малая_АСФ,V3_гильотина_АСФ
№,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",1295.6057,0.7961,9.5459,69790.2702,42.8848,514.2062,2008.1755,1066.1637,12783.7377,9.5535,514.6176,12793.9646


In [ ]:
preparation_df.head(2)

,наименование оборудования,полная,частичная,частичная_малая,частичная_свищ,площадь полная,площадь частичная,облако,твс,напор гильотины,напор малой,напор свища,безнапор гильотины,безнапор малой,безнапор свища,площадь_гильотины,площадь_малой,площадь_свища
№,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
2,"нефтепровод (1000х12, 5000 м)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001


In [ ]:
for i, r in df.iterrows():
  V_2_3_gil = masses_cv_gil_df.loc[i, 'V2_гильотина'] + masses_cv_gil_df.loc[i, 'V3_гильотина']
  preparation_df.loc[i, 'полная'] = masses_cv_gil_df.loc[i, 'V1_гильотина'] + (V_2_3_gil if V_2_3_gil < df.loc[i, 'масса_в_участке'] else df.loc[i, 'масса_в_участке'])
  V_2_3_sv = masses_cv_gil_df.loc[i, 'V2_свищ'] + masses_cv_gil_df.loc[i, 'V3_свищ']
  preparation_df.loc[i, 'частичная_свищ'] = masses_cv_gil_df.loc[i, 'V1_свищ'] + (V_2_3_sv if V_2_3_sv < df.loc[i, 'масса_в_участке'] else df.loc[i, 'масса_в_участке'])
  preparation_df.loc[i, 'частичная'] = masses_cv_gil_df.loc[i, 'V1_свищ'] + (V_2_3_sv if V_2_3_sv < df.loc[i, 'масса_в_участке'] else df.loc[i, 'масса_в_участке'])
  V_2_3_mal = masses_cv_gil_df.loc[i, 'V2_малая'] + masses_cv_gil_df.loc[i, 'V3_малая']
  preparation_df.loc[i, 'частичная_малая'] = masses_cv_gil_df.loc[i, 'V1_малая'] + (V_2_3_mal if V_2_3_mal < df.loc[i, 'масса_в_участке'] else df.loc[i, 'масса_в_участке'])
  

In [ ]:
preparation_df

,наименование оборудования,полная,частичная,частичная_малая,частичная_свищ,площадь полная,площадь частичная,облако,твс,напор гильотины,напор малой,напор свища,безнапор гильотины,безнапор малой,безнапор свища,площадь_гильотины,площадь_малой,площадь_свища
№,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",8666.7155,1305.9477,70347.3612,1305.9477,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
2,"нефтепровод (1000х12, 5000 м)",5337.4555,1301.1747,70090.2581,1301.1747,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
3,"нефтепровод (1000х12, 1000 м)",2674.0255,1297.3564,69884.5756,1297.3564,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
4,"нефтепровод (1000х12, 500 м)",2341.1055,1296.8791,69858.8653,1296.8791,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
5,"нефтепровод (1000х12, 100 м)",2074.7655,1296.4973,69838.2971,1296.4973,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
6,"нефтепровод (1000х12, 5 м)",2011.5055,1296.4066,69793.6002,1296.4066,NaN,NaN,NaN,NaN,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
7,"нефтепровод (720х12, 10000 м)",4407.3128,1305.9477,35773.9301,1305.9477,NaN,NaN,NaN,NaN,3404.076102,136.923731,4.998479,1807.263549,72.6944,2.653751,0.068102,0.002739,0.0001
8,"нефтепровод (720х12, 5000 м)",2714.2728,1301.1747,35643.1848,1301.1747,NaN,NaN,NaN,NaN,3404.076102,136.923731,4.998479,1807.263549,72.6944,2.653751,0.068102,0.002739,0.0001
9,"нефтепровод (720х12, 1000 м)",1359.8328,1297.3564,35538.5886,1297.3564,NaN,NaN,NaN,NaN,3404.076102,136.923731,4.998479,1807.263549,72.6944,2.653751,0.068102,0.002739,0.0001


In [ ]:
preparation_df['площадь полная'] = Strait_area(preparation_df['полная'], ro, df['Коэффициент подстил поверхности'])

In [ ]:
preparation_df['площадь частичная'] = Strait_area(preparation_df['частичная'], ro, df['Коэффициент подстил поверхности'])

In [ ]:
preparation_df['облако'] = Evaporation_404() * 3600 * preparation_df['площадь полная']

In [ ]:
preparation_df['твс'] = preparation_df['облако'] * 0.1

In [ ]:
preparation_df.head(3)

,наименование оборудования,полная,частичная,частичная_малая,частичная_свищ,площадь полная,площадь частичная,облако,твс,напор гильотины,напор малой,напор свища,безнапор гильотины,безнапор малой,безнапор свища,площадь_гильотины,площадь_малой,площадь_свища
№,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",8666.7155,1305.9477,70347.3612,1305.9477,48689.412921,7336.784831,20370.300315,2037.030032,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
2,"нефтепровод (1000х12, 5000 м)",5337.4555,1301.1747,70090.2581,1301.1747,29985.705056,7309.970225,12545.187557,1254.518756,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001
3,"нефтепровод (1000х12, 1000 м)",2674.0255,1297.3564,69884.5756,1297.3564,15022.615169,7288.519101,6285.045642,628.504564,6693.918444,269.252585,4.998479,3553.879068,142.949326,2.653751,0.133919,0.005387,0.0001


##С-2 Сценарий пожар-пролива

In [ ]:
scenario_c2_df = pd.DataFrame(columns=['10,5', '7,0','1,4'])

In [ ]:
scenario_c2_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c2_df.insert(1,'площадь пожара', preparation_df['площадь полная'] )

In [ ]:
for index, row in scenario_c2_df.iterrows():
  result = StraitFire_404(row[1])
  for key, value in result.items():
    scenario_c2_df.loc[index, key] = value

In [ ]:
scenario_c2_df.head()

,наименование оборудования,площадь пожара,"10,5","7,0","1,4"
№,,,,,
1,"нефтепровод (1000х12, 10000 м)",48689.412921,320.0,386.3,689.1
2,"нефтепровод (1000х12, 5000 м)",29985.705056,259.6,315.3,573.9
3,"нефтепровод (1000х12, 1000 м)",15022.615169,192.1,235.4,439.6
4,"нефтепровод (1000х12, 500 м)",13152.27809,181.3,222.5,417.3
5,"нефтепровод (1000х12, 100 м)",11655.985955,171.9,211.3,398.0


##С-3 Сценарий пожар вспышка

In [ ]:
scenario_c3_df = pd.DataFrame(columns=['вспышка'])

In [ ]:
scenario_c3_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c3_df['вспышка'] = fire_flash_404(preparation_df['облако'])

##С-4 Сценарий Взрыв ТВС

In [ ]:
scenario_c4_df = pd.DataFrame(columns=['100', '53','28', '12', '5', '3'])

In [ ]:
scenario_c4_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c4_df.insert(1,'масса', preparation_df['твс'] )

In [ ]:
for index, row in scenario_c4_df.iterrows():
  result = TVS_404(row[1])
  for key, value in result.items():
    scenario_c4_df.loc[index, key] = value

In [ ]:
scenario_c4_df

,наименование оборудования,масса,100,53,28,12,5,3
№,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",2037.030032,-,-,-,101.6,274.7,469.2
2,"нефтепровод (1000х12, 5000 м)",1254.518756,-,-,-,70.1,196.3,337.3
3,"нефтепровод (1000х12, 1000 м)",628.504564,-,-,-,-,121.0,210.2
4,"нефтепровод (1000х12, 500 м)",550.254847,-,-,-,35.0,110.2,191.8
5,"нефтепровод (1000х12, 100 м)",487.654133,-,-,-,31.1,101.1,176.5
6,"нефтепровод (1000х12, 5 м)",472.785464,-,-,-,-,98.9,172.8
7,"нефтепровод (720х12, 10000 м)",1035.897455,-,-,-,60.2,171.8,296.0
8,"нефтепровод (720х12, 5000 м)",637.964314,-,-,-,40.1,122.3,212.3
9,"нефтепровод (720х12, 1000 м)",319.61592,-,-,-,-,74.7,131.8


##Определение вероятности реализации событий

In [ ]:
frequencies_df = pd.DataFrame(columns=['полная','частичная',
                                       'С-1А(п)', 'С-1А(ч)', 'С-2', 'С-3', 'С-4' ]) 

In [ ]:
frequencies_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

Расчитываем вероятность возникновения иницирующих событий с учетом длины трубопроводов

In [ ]:
for index, row in df.iterrows():
  result = frequencies_pipline(row['диаметр']) 
  for key, value in result.items():
    frequencies_df.loc[index, key] = value * row['Протяженность, м']

Расчитываем вероятность возникновения различных сценариев

In [ ]:
for index, row in frequencies_df.iterrows():
  
   frequencies_df.loc[index, 'С-1А(п)'] = row['полная'] * scenario_lvz['полная']['пролив']
   frequencies_df.loc[index, 'С-1А(ч)'] = row['частичная'] * scenario_lvz['частичная']['пролив']
   frequencies_df.loc[index, 'С-2'] = row['полная'] * scenario_lvz['полная']['пожар']
   frequencies_df.loc[index, 'С-3'] = row['полная'] * scenario_lvz['полная']['вспышка']
   frequencies_df.loc[index, 'С-4'] = row['полная'] * scenario_lvz['полная']['ТВС'] 

In [ ]:
frequencies_df.head()

,наименование оборудования,полная,частичная,С-1А(п),С-1А(ч),С-2,С-3,С-4
№,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",0.005,0.001,0.00304,0.00093,0.001,0.000384,0.000576
2,"нефтепровод (1000х12, 5000 м)",0.0025,0.0005,0.00152,0.000465,0.0005,0.000192,0.000288
3,"нефтепровод (1000х12, 1000 м)",0.0005,0.0001,0.000304,0.000093,0.0001,0.000038,0.000058
4,"нефтепровод (1000х12, 500 м)",0.00025,0.00005,0.000152,0.000047,0.00005,0.000019,0.000029
5,"нефтепровод (1000х12, 100 м)",0.00005,0.00001,0.00003,0.000009,0.00001,0.000004,0.000006


#Заполнение данных в Excel

##Количество вещества в оборудовании

In [ ]:
wb_rasp = wb['Rasp']

In [ ]:
for i, r in df.iterrows():
  wb_rasp.cell(row = i + 3, column = 1, value = df['Блок'][i])
  wb_rasp.cell(row = i + 3, column = 2, value = f'''{df['наименование оборудования'][i]}, {df['Вещество'][i]}''')
  wb_rasp.cell(row = i + 3, column = 3, value = f'''{df['Протяженность, м'][i]}, м''')
  wb_rasp.cell(row = i + 3, column = 4, value = f'''{str(df['масса_в_участке'][i]).replace('.', ',')}''')
  wb_rasp.cell(row = i + 3, column = 4).number_format = '0,00'
  wb_rasp.cell(row = i + 3, column = 6, value = f'''Жидкость''')
  wb_rasp.cell(row = i + 3, column = 7, value = f'''{df['давление'][i]}''')
  wb_rasp.cell(row = i + 3, column = 8, value = f'''{df['Температура'][i]}''') 
wb_rasp.cell(row = 4, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=4, start_column=5, end_row=len(df)+3, end_column=5)
#итог
wb_rasp.cell(row = len(df)+4, column = 1, value = 'Итого опасного вещества - Нефть,т')
wb_rasp.cell(row = len(df)+4, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=len(df)+4, start_column=1, end_row=len(df)+4, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+4, start_column=5, end_row=len(df)+4, end_column=8)
wb_rasp.cell(row = len(df)+5, column = 1, value = 'из них - в сосудах (аппаратах), т')
wb_rasp.cell(row = len(df)+5, column = 5, value = '0')
wb_rasp.merge_cells(start_row=len(df)+5, start_column=1, end_row=len(df)+5, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+5, start_column=5, end_row=len(df)+5, end_column=8)
wb_rasp.cell(row = len(df)+6, column = 1, value = 'в трубопроводах, т')
wb_rasp.cell(row = len(df)+6, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=len(df)+6, start_column=1, end_row=len(df)+6, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+6, start_column=5, end_row=len(df)+6, end_column=8)

##Массы по сценариям

In [ ]:
wb_mass = wb['Masses']

In [ ]:
j = 4
for i, r in preparation_df.iterrows():
  wb_mass.cell(row = j, column = 1, value = 'С-1А')
  wb_mass.cell(row = j + 1, column = 1, value = 'С-2')
  wb_mass.cell(row = j + 2, column = 1, value = 'С-3')
  wb_mass.cell(row = j + 3, column = 1, value = 'С-4А')
  #column 2
  wb_mass.cell(row = j, column = 2, value = f'''{preparation_df['наименование оборудования'][i]}''')
  wb_mass.merge_cells(start_row=j, start_column=2, end_row=j+3, end_column=2)
  #column 3
  wb_mass.cell(row = j, column = 3, value = 'Пролив нефти на открытой площадке')
  wb_mass.cell(row = j + 1, column = 3, value = 'Пожар пролива на открытой площадке')
  wb_mass.cell(row = j + 2, column = 3, value = 'Пожар-вспышка на открытой площадке')
  wb_mass.cell(row = j + 3, column = 3, value = 'Взрыв ГПВС на открытой площадке') 
  #column 4
  wb_mass.cell(row = j, column = 4, value = 'Экологическое загрязнение')
  wb_mass.cell(row = j + 1, column = 4, value = 'Открытое пламя, тепловое излучение')
  wb_mass.cell(row = j + 2, column = 4, value = 'Высокотемпературные продукты сгорания')
  wb_mass.cell(row = j + 3, column = 4, value = 'Ударная волна, осколки')
  #column 5
  wb_mass.cell(row = j, column = 5, value = f'''{round(preparation_df['полная'][i], 2)} / {round(preparation_df['частичная'][i],2)}''')
  wb_mass.merge_cells(start_row=j, start_column=5, end_row=j+3, end_column=5)
  #column 6
  wb_mass.cell(row = j, column = 6, value = f'''{round(preparation_df['полная'][i], 2)} / {round(preparation_df['частичная'][i],2)}''')
  wb_mass.cell(row = j + 1, column = 6, value = round(preparation_df['полная'][i], 2))
  wb_mass.cell(row = j + 2, column = 6, value = round(preparation_df['облако'][i], 2))
  wb_mass.cell(row = j + 3, column = 6, value = round(preparation_df['твс'][i], 2))  
  j = j + 4
 

##Напор

In [ ]:
wb_napor = wb['Napor']

In [ ]:
masses_cv_gil_df.head(1)

,наименование оборудования,V1_свищ,V2_свищ,V3_свищ,V1_малая,V2_малая,V3_малая,V1_гильотина,V2_гильотина,V3_гильотина,V3_свищ_АСФ,V3_малая_АСФ,V3_гильотина_АСФ
№,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",1295.6057,0.7961,9.5459,69790.2702,42.8848,514.2062,2008.1755,1066.1637,12783.7377,9.5535,514.6176,12793.9646


In [ ]:
for i, r in masses_cv_gil_df.iterrows():
  wb_napor.cell(row = i + 1, column = 1, value = masses_cv_gil_df.loc[i, 'наименование оборудования'])
  wb_napor.cell(row = i + 1, column = 2, value = masses_cv_gil_df.loc[i, 'V1_свищ'])
  wb_napor.cell(row = i + 1, column = 3, value = masses_cv_gil_df.loc[i, 'V2_свищ'])
  wb_napor.cell(row = i + 1, column = 4, value = masses_cv_gil_df.loc[i, 'V3_свищ'])
  wb_napor.cell(row = i + 1, column = 5, value = masses_cv_gil_df.loc[i, 'V1_малая'])
  wb_napor.cell(row = i + 1, column = 6, value = masses_cv_gil_df.loc[i, 'V2_малая'])
  wb_napor.cell(row = i + 1, column = 7, value = masses_cv_gil_df.loc[i, 'V3_малая'])
  wb_napor.cell(row = i + 1, column = 8, value = masses_cv_gil_df.loc[i, 'V1_гильотина'])
  wb_napor.cell(row = i + 1, column = 9, value = masses_cv_gil_df.loc[i, 'V2_гильотина'])
  wb_napor.cell(row = i + 1, column = 10, value = masses_cv_gil_df.loc[i, 'V3_гильотина'])
  wb_napor.cell(row = i + 1, column = 11, value = masses_cv_gil_df.loc[i, 'V3_свищ_АСФ'])
  wb_napor.cell(row = i + 1, column = 12, value = masses_cv_gil_df.loc[i, 'V3_малая_АСФ'])
  wb_napor.cell(row = i + 1, column = 13, value = masses_cv_gil_df.loc[i, 'V3_гильотина_АСФ'])

##Сценарий С-1

In [ ]:
wb_c1 = wb['C1']

In [ ]:
j = 4
for i, r in preparation_df.iterrows():
  #column 1
  wb_c1.cell(row = j, column = 1, value = 'С-1А(п)')
  wb_c1.merge_cells(start_row=j, start_column=1, end_row=j+1, end_column=1)
  wb_c1.cell(row = j + 2, column = 1, value = 'С-1А(ч)')
  wb_c1.merge_cells(start_row=j+2, start_column=1, end_row=j+3, end_column=1)
  #column 2
  wb_c1.cell(row = j, column = 2, value = f'''{preparation_df['наименование оборудования'][i]}''')
  wb_c1.merge_cells(start_row=j, start_column=2, end_row=j+3, end_column=2) 
  #column 3
  wb_c1.cell(row = j, column = 3, value = 'Площадь пролива, м2')
  wb_c1.cell(row = j + 1, column = 3, value = 'Радиус пролива, м')
  wb_c1.cell(row = j + 2, column = 3, value = 'Площадь пролива, м2')
  wb_c1.cell(row = j + 3, column = 3, value = 'Радиус пролива, м')
  #column 4
  wb_c1.cell(row = j, column = 4, value = round(preparation_df['площадь полная'][i], 2))
  wb_c1.cell(row = j + 1, column = 4, value = round(preparation_df['площадь полная'][i] / pi, 2))
  wb_c1.cell(row = j + 2, column = 4, value = round(preparation_df['площадь частичная'][i], 2))
  wb_c1.cell(row = j + 3, column = 4, value = round(preparation_df['площадь частичная'][i] / pi, 2))   
  j += 4


##Сценарий С-2

In [ ]:
wb_c2 = wb['C2']

In [ ]:
j = 4
for i, r in scenario_c2_df.iterrows():
  #column 1
  wb_c2.cell(row = j, column = 1, value = scenario_c2_df['наименование оборудования'][i])
  wb_c2.merge_cells(start_row=j, start_column=1, end_row=j+4, end_column=1)
  #column 2
  wb_c2.cell(row = j, column = 2, value = 'Площадь пожара, м2')
  wb_c2.cell(row = j+1, column = 2, value = 'Радиусы зон поражения тепловым излучением, м:')
  wb_c2.cell(row = j+2, column = 2, value = '– непереносимая боль через 3-5 с (10,5 кВт/м2)')
  wb_c2.cell(row = j+3, column = 2, value = '– непереносимая боль через 20-30 с (7,0 кВт/м2)')
  wb_c2.cell(row = j+4, column = 2, value = '– зона отсутствия негативных последствий (1,4 кВт/м2)')  
  #column 3
  wb_c2.cell(row = j, column = 3, value = round(scenario_c2_df['площадь пожара'][i], 2))
  wb_c2.cell(row = j + 2, column = 3, value = scenario_c2_df['10,5'][i])
  wb_c2.cell(row = j + 3, column = 3, value = scenario_c2_df['7,0'][i])
  wb_c2.cell(row = j + 4, column = 3, value = scenario_c2_df['1,4'][i])   
  j += 5

##Сценарий С-3

In [ ]:
wb_c3 = wb['C3']

In [ ]:
j = 4
for i, r in scenario_c3_df.iterrows():
  #column 1
  wb_c3.cell(row = j, column = 1, value = scenario_c3_df['наименование оборудования'][i])
  #column 2
  wb_c3.cell(row = j, column = 2, value = 'Радиус зоны воздействия высокотемпературных продуктов сгорания, м') 
  #column 3
  wb_c3.cell(row = j, column = 3, value = round(scenario_c3_df['вспышка'][i], 2))  
  j += 1

## Сценарий С-4

In [ ]:
wb_c4 = wb['C4']

In [ ]:
j = 4
for i, r in scenario_c4_df.iterrows():
  #column 1
  wb_c4.cell(row = j, column = 1, value = scenario_c4_df['наименование оборудования'][i])
  #column 2
  wb_c4.cell(row = j, column = 2, value = scenario_c4_df['100'][i]) 
  #column 3
  wb_c4.cell(row = j, column = 3, value = scenario_c4_df['53'][i]) 
  #column 4
  wb_c4.cell(row = j, column = 4, value = scenario_c4_df['28'][i]) 
  #column 5
  wb_c4.cell(row = j, column = 5, value = scenario_c4_df['12'][i])
  #column 6
  wb_c4.cell(row = j, column = 6, value = scenario_c4_df['5'][i])
  #column 7
  wb_c4.cell(row = j, column = 7, value = scenario_c4_df['3'][i])
  j += 1

##Вероятность реализации

In [ ]:
chast_tr = wb['Chast_tr']

In [ ]:
j = 3
for i, r in frequencies_df.iterrows():
  #column 1
  chast_tr.cell(row = j, column = 1, value = 'С-1А(ч)')
  chast_tr.cell(row = j+1, column = 1, value = 'С-1А(п)')
  chast_tr.cell(row = j+2, column = 1, value = 'С-2')
  chast_tr.cell(row = j+3, column = 1, value = 'С-3')
  chast_tr.cell(row = j+4, column = 1, value = 'С-4')
  #column 2
  chast_tr.cell(row = j, column = 2, value = frequencies_df['наименование оборудования'][i])
  chast_tr .merge_cells(start_row=j, start_column=2, end_row=j+4, end_column=2)
  #column 3  
  chast_tr.cell(row = j, column = 3, value = frequencies_df['С-1А(ч)'][i])
  chast_tr.cell(row = j+1, column = 3, value = frequencies_df['С-1А(п)'][i])
  chast_tr.cell(row = j+2, column = 3, value = frequencies_df['С-2'][i])
  chast_tr.cell(row = j+3, column = 3, value = frequencies_df['С-3'][i])
  chast_tr.cell(row = j+4, column = 3, value = frequencies_df['С-4'][i])
  j += 5


##Сохранение

In [ ]:
wb.save('/content/drive/MyDrive/Work, Study, Life/Analyst/Pipeline_method/result.xlsx')

https://docs.google.com/spreadsheets/d/1w_QSPteFTJeMEs52QoI4aqBZuaj-34MG/edit?usp=share_link&ouid=106885717380627810664&rtpof=true&sd=true